## load wrong data

In [15]:
import os
import re
import numpy as np
import pandas as pd

In [2]:
wrong_file = '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong_1.txt'

In [3]:
with open(wrong_file,'r') as f:
    wrong_list = f.readlines()
len(wrong_list)

In [5]:
uid = []
iid = []
his = [] 
label = []
his_label = []
his_title = []
title = []
for line in wrong_list:
    data = line.split('\sep')
    uid.append(int(data[0]))
    iid.append(int(data[1]))
    his.append(eval(data[2]))
    label.append(int(data[3]))
    his_label.append(eval(data[4]))
    his_title.append(data[5].replace('with feature <ItemID> ',''))
    title.append(data[6].replace('\n',''))

In [6]:
wrong_data = pd.DataFrame({"uid":uid,'iid':iid,'label':label, 'his':his,'his_label':his_label,'his_title':his_title,'title':title})

In [7]:
wrong_data.head()

,uid,iid,label,his,his_label,his_title,title
0,20927,40012,0,"[30252, 44599, 36606, 44751, 45031, 44960, 187...","[0, 1, 1, 1, 1, 1, 1, 0, 1, 1]","""The Palisade (Lavender Shores)"" (evaluation: ...","""P.S. I Spook You"" with description: life, wri..."
1,8483,20626,1,"[40433, 20414, 20471, 33539, 39841, 34463, 347...","[1, 1, 0, 1, 0, 0, 0, 0, 1, 0]","""The Bad Boy Next Door"" (evaluation: Yes) with...","""The Bad Luck Bride: The Cavensham Heiresses"" ..."
2,6390,3886,1,"[20334, 10160, 9536, 37380, 15208, 19818]","[1, 0, 0, 1, 0, 1]","""Wintersong: A Novel"" (evaluation: Yes) with d...","""Greater Gotham: A History of New York City fr..."
3,5639,21757,1,"[40397, 42989, 36136, 2032, 23311, 38134, 2307...","[1, 0, 0, 1, 1, 1, 1, 1, 1, 1]","""Accidentally Compromising the Duke"" (evaluati...","""It Started with a Kiss (The Worthingtons)"" wi..."
4,15532,45963,0,"[33547, 33717, 33775, 43474, 40343, 33677, 169...","[1, 0, 1, 0, 1, 1, 0, 1, 1, 1]","""Trick (A Cocky Cage Fighter Novel) (Volume 7)...","""The Bartender (Modern Love) (Volume 1)"" with ..."


In [8]:
wrong_data['his_pad'] = wrong_data['his'].map(lambda x: [0]+x if len(x) < 10 else x)
wrong_data['his_label_pad'] = wrong_data['his_label'].map(lambda x: [0]+x if len(x) < 10 else x)

In [9]:
wrong_data = wrong_data[['uid','iid','label','his_pad','his_label_pad','his_title','title']]
wrong_data.columns = ['uid','iid','label','his','his_label','his_title','title']
wrong_data.head()

,uid,iid,label,his,his_label,his_title,title
0,20927,40012,0,"[30252, 44599, 36606, 44751, 45031, 44960, 187...","[0, 1, 1, 1, 1, 1, 1, 0, 1, 1]","""The Palisade (Lavender Shores)"" (evaluation: ...","""P.S. I Spook You"" with description: life, wri..."
1,8483,20626,1,"[40433, 20414, 20471, 33539, 39841, 34463, 347...","[1, 1, 0, 1, 0, 0, 0, 0, 1, 0]","""The Bad Boy Next Door"" (evaluation: Yes) with...","""The Bad Luck Bride: The Cavensham Heiresses"" ..."
2,6390,3886,1,"[0, 20334, 10160, 9536, 37380, 15208, 19818]","[0, 1, 0, 0, 1, 0, 1]","""Wintersong: A Novel"" (evaluation: Yes) with d...","""Greater Gotham: A History of New York City fr..."
3,5639,21757,1,"[40397, 42989, 36136, 2032, 23311, 38134, 2307...","[1, 0, 0, 1, 1, 1, 1, 1, 1, 1]","""Accidentally Compromising the Duke"" (evaluati...","""It Started with a Kiss (The Worthingtons)"" wi..."
4,15532,45963,0,"[33547, 33717, 33775, 43474, 40343, 33677, 169...","[1, 0, 1, 0, 1, 1, 0, 1, 1, 1]","""Trick (A Cocky Cage Fighter Novel) (Volume 7)...","""The Bartender (Modern Love) (Volume 1)"" with ..."


In [10]:
wrong_data.to_csv(wrong_file.replace('.txt','.csv'),index=False)

## Get analysis

In [2]:
import os
import numpy as np
import pandas as pd

In [56]:
wrong_data = pd.read_csv('/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong_1.csv')

In [57]:
from tqdm import tqdm
import requests
from openai import OpenAI
url = 'https://api.siliconflow.cn/v1/'
api_key = 'sk-aqjpdmbxfmztkooxohnyxcbngysqhpgmjwxojihwwzuxojnj'

def get_response(client,message):
    for delay_secs in (2**x for x in range(0, 10)):
        try:
            response = client.chat.completions.create(
                model="Qwen/QwQ-32B",
                messages=message,
                stream=False, 
                max_tokens=1024
            )
            content = response.choices[0].message.content
            reasoning_content = response.choices[0].message.reasoning_content
            return content, reasoning_content
        except openai.OpenAIError as e:
            randomness_collision_avoidance = random.randint(0, 1000) / 1000.0
            sleep_dur = delay_secs + randomness_collision_avoidance
            print(f"Error: {e}. Retrying in {round(sleep_dur, 2)} seconds.")
            time.sleep(sleep_dur)
            continue

In [58]:
label2text = {0:'No',1:'Yes'}
prompt_template = '''A user has given high ratings to the following books: <HisItemList>. Using all available information, make a prediction about whether the user would enjoy the book titled <TargetItemTitle>?'''
hints = 'The correct response is <answer>. Reflect on multiple aspects based on historical information and reanalyze to determine if the target item will be liked by users. Explain the reason for the oversight based on the previous analysis.'

In [62]:
reasons = []
sbar = tqdm(total = len(wrong_data))
sbar.set_description('Generating Reasoning')
for idx in range(len(wrong_data)):
    row = wrong_data.iloc[idx]
    label = row['label']
    his_seq = row['his_title']
    target = row['title']
    message = prompt_template.replace('<HisItemList>',his_seq).replace('<TargetItemTitle>',target)
    his_message = [
        {"role": "system","content": "You are a helpful recommender answer with Yes or No."},
        {"role": "user","content": message}
    ]
    client = OpenAI(base_url=url,api_key=api_key)
    answer,reasoning_content = get_response(client,his_message)
    cnt = 1
    while label2text[label] not in answer:
        # his_message.pop(0)
        his_message.append({"role": "assistant","content": label2text[1-label]})
        his_message.append({"role": "user","content": hints.replace('<answer>',label2text[label])})
        answer,reasoning_content = get_response(client,his_message)
        cnt += 1
    reasons.append(f'{label2text[label]}. {reasoning_content}')
    sbar.set_postfix(cnt=cnt)
    if cnt > 5:
        print(f'cnt:{cnt}')
    sbar.update(1)
sbar.close()

Generating Reasoning:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Reasoning: 100%|██████████| 2/2 [01:14<00:00, 37.32s/it, cnt=2]


In [ ]:
wrong_data['reason'] = reasons

In [ ]:
wrong_data.to_csv(wrong_file.replace('.txt','.csv'),index=False)

## Merge csv

In [1]:
import os
import pandas as pd
import numpy as np
import re
from glob import glob

In [2]:
csv_dir = '/home/yuqihang/workroot/models/data/collm/yelpdu/reason'

In [3]:
csv_files = glob(f'{csv_dir}/wrong*.csv')

In [4]:
csv_files

['/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/wrong_batch_15.csv',
 '/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/wrong_batch_17.csv',
 '/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/wrong_batch_7.csv',
 '/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/wrong_batch_14.csv',
 '/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/wrong_batch_1.csv',
 '/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/wrong_batch_11.csv',
 '/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/wrong_batch_3.csv',
 '/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/wrong_batch_13.csv',
 '/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/wrong_batch_5.csv',
 '/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/wrong_batch_8.csv',
 '/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/wrong_batch_6.csv',
 '/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/wrong_batch_18.csv',
 '/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/

In [5]:
save_file = 'reason.csv'

In [6]:
dfs = []
for csv_file in csv_files:
    file_path = csv_file#os.path.join(csv_dir, csv_file)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        if not dfs:
            dfs.append(df)
        else:
            if list(df.columns) == list(dfs[0].columns):
                dfs.append(df)
            else:
                print(f"表头不一致: {csv_file}")
    else:
        print(f"文件不存在: {csv_file}")
if dfs:
    df = pd.concat(dfs, ignore_index=True)
else:
    print("没有有效的文件可以合并")

In [7]:
def convert_str_2_list(x):
    return np.array([int(item) for item in re.split(r'\s+', x[1:-1].replace(',',' ').strip(' '))])
def dftype(df):
    for idx in df.columns:
        if isinstance(df[idx][0],list) or isinstance(df[idx][0],np.ndarray):
            print(idx,type(df[idx][0]),type(df[idx][0][0]))
        else:
            print(idx,type(df[idx][0]))

In [9]:
df['his_label'] = df['his_label'].map(lambda x: convert_str_2_list(x))
df['his'] = df['his'].map(lambda x: convert_str_2_list(x))

In [12]:
df['reason'] = df['reason'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))

In [10]:
dftype(df)

uid <class 'numpy.int64'>
iid <class 'numpy.int64'>
label <class 'numpy.int64'>
his <class 'numpy.ndarray'> <class 'numpy.int64'>
his_label <class 'numpy.ndarray'> <class 'numpy.int64'>
his_title <class 'str'>
title <class 'str'>
reason <class 'str'>


In [13]:
df.head()

,uid,iid,label,his,his_label,his_title,title,reason
0,2275,531,1,"[2696, 2081, 341, 1774, 2933, 3132, 1062, 3576...","[1, 1, 0, 0, 0, 1, 1, 0, 0, 0]","""Love & Sex"" (evaluation: Yes) with descriptio...","""Hurricane, The"" with description: Drama","Yes. Okay, let's see. The user wants me to pre..."
1,2275,900,0,"[3285, 2696, 2081, 341, 1774, 2933, 3132, 1062...","[0, 1, 1, 0, 0, 0, 1, 1, 0, 0]","""Eight Days a Week"" (evaluation: No) with desc...","""Big Momma's House"" with description: Comedy","No. Alright, let's tackle this problem step by..."
2,2279,185,1,"[647, 848, 738, 559, 1221, 1096, 6, 1345, 642,...","[1, 0, 0, 0, 0, 0, 0, 1, 0, 1]","""Star Trek IV: The Voyage Home"" (evaluation: Y...","""Being John Malkovich"" with description: Comedy","Yes. Okay, let me try to figure this out. So, ..."
3,2279,2092,1,"[999, 647, 848, 738, 559, 1221, 1096, 6, 1345,...","[1, 1, 0, 0, 0, 0, 0, 0, 1, 0]","""Metropolis"" (evaluation: Yes) with descriptio...","""Wonder Boys"" with description: Comedy, Drama","Yes. Okay, let me try to work through this ste..."
4,2296,1196,1,"[1119, 1150, 2379, 264, 1180, 483, 1617, 302, ...","[0, 0, 0, 1, 0, 1, 1, 0, 1, 1]","""Scream"" (evaluation: No) with description: Ho...","""Some Like It Hot"" with description: Comedy, C...","Yes. Okay, let me try to figure this out. The ..."


In [14]:
if len(save_file) == 0:
    save_ids = [csv_file.split('.')[0].split('_')[1:] for csv_file in csv_files]
    save_path = f'{csv_dir}/wrong_{"_".join(sorted(sum(save_ids,[])))}.csv'
else:
    save_path = f'{csv_dir}/{save_file}'
if os.path.exists(save_path):
    print(f'{save_path} already exists, skip')
else:
    df.to_csv(save_path,index=False)
save_path,len(df)

('/home/yuqihang/workroot/models/data/collm/ml1mdu/reason/reason.csv', 1000)

In [15]:
len(df),dftype(df)

uid <class 'numpy.int64'>
iid <class 'numpy.int64'>
label <class 'numpy.int64'>
his <class 'numpy.ndarray'> <class 'numpy.int64'>
his_label <class 'numpy.ndarray'> <class 'numpy.int64'>
his_title <class 'str'>
title <class 'str'>
reason <class 'str'>


(1000, None)

## Convert to training data

In [37]:
import re
import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv('/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/reason.csv')

In [38]:
df.head(2)

,uid,iid,label,his,his_label,his_title,title,reason
0,11630,20516,0,"[12376, 17251, 19283, 26190, 42681, 43495, 101...","[1, 1, 0, 1, 1, 1, 1, 1, 1, 0]","""Lanlan 3x3x2 Speed Brain Teaser Cube Puzzle, ...","""New!! Shengshou 10x10x10 Speed Cube Puzzle 10...","No. Okay, let's see. The user has rated severa..."
1,11639,46199,1,"[28323, 28875, 31560, 33108, 31681, 32123, 399...","[1, 0, 0, 1, 1, 1, 1, 1, 1, 1]","""My Little Pony Equestria Girls Rainbow Rocks ...","""Play-Doh Doctor Drill 'n Fill Set"" with descr...","Yes. Okay, let me try to figure this out again..."


In [39]:
len(df)

1600

In [16]:
df['not_cold'] = pd.DataFrame(np.ones(df.shape[0]),index=df.index).astype("int")

In [17]:
wrong = df[['uid','iid','label','his','his_label','not_cold','reason']]

In [18]:
wrong.head(2)

,uid,iid,label,his,his_label,not_cold,reason
0,2275,531,1,"[2696, 2081, 341, 1774, 2933, 3132, 1062, 3576...","[1, 1, 0, 0, 0, 1, 1, 0, 0, 0]",1,"Yes. Okay, let's see. The user wants me to pre..."
1,2275,900,0,"[3285, 2696, 2081, 341, 1774, 2933, 3132, 1062...","[0, 1, 1, 0, 0, 0, 1, 1, 0, 0]",1,"No. Alright, let's tackle this problem step by..."


In [19]:
dftype(wrong)

uid <class 'numpy.int64'>
iid <class 'numpy.int64'>
label <class 'numpy.int64'>
his <class 'numpy.ndarray'> <class 'numpy.int64'>
his_label <class 'numpy.ndarray'> <class 'numpy.int64'>
not_cold <class 'numpy.int64'>
reason <class 'str'>


In [20]:
len(wrong)

1000

In [21]:
wrong.to_pickle('/home/yuqihang/workroot/models/data/collm/ml1mdu/reason_ood2.pkl')